In [ ]:
import os
import datasets, transformers

from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AutoTokenizer

import pandas as pd
import numpy as np

os.environ["WANDB_DISABLED"] = "true"

In [ ]:
class CFG:
    input_path = '../input/us-patent-phrase-to-phrase-matching/'
    model_path = '../input/xlm-roberta-large-fold4/uspppm_4/'


In [ ]:
titles = pd.read_csv('../input/cpc-codes/titles.csv')

test = pd.read_csv(f"{CFG.input_path}test.csv")
test = test.merge(titles, left_on='context', right_on='code')
test['input'] = test['title']+'[SEP]'+test['anchor']
test = test.drop(columns=["context", "code", "class", "subclass", "group", "main_group", "anchor", "title", "section"])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(CFG.model_path)
    
def process_test(unit):
    return {
        **tokenizer( unit['input'], unit['target'])
    }

test_ds = datasets.Dataset.from_pandas(test)
test_ds = test_ds.map(process_test, remove_columns=['id', 'target', 'input', '__index_level_0__'])

model = AutoModelForSequenceClassification.from_pretrained(CFG.model_path, 
                                                           num_labels=1)
trainer = Trainer(
        model,
        tokenizer=tokenizer,
    )

outputs = trainer.predict(test_ds)
prediction = outputs.predictions.reshape(-1)

# np.clip(prediction, 0.0, 1.0, out=prediction)

submission = datasets.Dataset.from_dict({
    'id': test['id'],
    'score': prediction,
})

submission.to_csv('submission.csv', index=False)